This notebook creates a ``data_preprocessor`` class which converts the new incoming logs to the format that is acceptable by our machine learning model. The following are the operations that will be performed on the incoming data:-
1. Converting the date columns to datetime format
2. Calculate mean ``impression_time`` for a particular user
3. Encode ``os_version``
4. Count unique apps used by the user
5. Extract hour and minute from ``impression_time``
6. Count unique user for an app
7. Calculating the counts of ``user_id`` and ``app_code``
8. Checking how many times user has clicked the ad and what was the last time he had an impression
9. Checking how many times user has clicked the ad from  A PARTICULAR APP and what was the last time he had an impression
10. In `view_aggdf`, encode the device type
11. merge `view_aggdf` and `train_df`